**Important** - There are 3 typical read modes and the default read mode is permissive.
1. permissive — All fields are set to null and corrupted records are placed in a string column called _corrupt_record
2. dropMalformed — Drops all rows containing corrupt records.
3. failFast — Fails when corrupt records are encountered.
**CSV Advanced Feature**
Very Important, Important, Not Important (just try to know once for all)
**Very Important**: path, schema, sep, header, inferSchema, samplingRatio, mode, columnNameOfCorruptRecord
**Important**: quote, escape,comment, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSpace, nullValue, nanValue, dateFormat, timestampFormat, multiLine, enforceSchema,pathGlobFilter, recursiveFileLookup,modifiedBefore,modifiedAfter
**Not Important**: encoding, positiveInf, negativeInf, maxColumns, maxCharsPerColumn, maxMalformedLogPerPartition, charToEscapeQuoteEscaping, emptyValue, locale, lineSep, unescapedQuoteHandling

In [0]:
df1=spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/malformeddata.txt",header=False,sep=",",comment='#')#permit malformed records and allow me to do a RCA (Root cause analysis)
display(df1)
df1.where("upper(_c2)=lower(_c2)").show()

In [0]:

struct1="cid int,name string,amt decimal(10,2),dop date"
df1=spark.read.format(struct1).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/malformeddata.txt",header=False,sep=",",comment='#',mode="PERMISSIVE") #Default mode is permissive
display(df1)
##Blindly drop malformed records
struct1="cid int,name string,amt decimal(10,2),dop date"
df1=spark.read.format(struct1).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/malformeddata.txt",header=False,sep=",",comment='#',mode="dropMalformed")  
display(df1)



 

In [0]:
 
struct1="cid int,name string,amt decimal(10,2),dop date,corrupt_record string"
df1=spark.read.format(struct1).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/malformed_data.txt",header=True,sep=",",comment='#',mode="permissive",columnNameOfCorruptRecord="corrupt_record") 
display(df1)


In [0]:
struct1="cid int,name string,amt decimal(10,2),dop date"
df1=spark.read.schema(struct1).csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/malformeddata1.txt",header=False,sep=","
,comment='#',quote="'",escape='~',ignoreLeadingWhiteSpace=True,ignoreTrailingWhiteSpace=True,dateFormat='yyyy-dd-MM',mode="permissive",modifiedAfter='2025-12-20',multiLine=True,nullValue='na')
display(df1)

JSON Advanced Feature -
**Very Important**- path,schema,columnNameOfCorruptRecord,dateFormat,timestampFormat,multiLine,pathGlobFilter,recursiveFileLookup
No header, No inferSchema, No sep in json...
**Important** - primitivesAsString(consider all cols as string), prefersDecimal, allowComments, allowUnquotedFieldNames, allowSingleQuotes, lineSep, samplingRatio, dropFieldIfAllNull, modifiedBefore, modifiedAfter, useUnsafeRow(This is performance optimization when the data is loaded into spark memory)
**Not Important** (just try to know once for all) - allowNumericLeadingZero, allowBackslashEscapingAnyCharacter, allowUnquotedControlChars, encoding, locale, allowNonNumericNumbers

In [0]:

df_json=spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/simple_json.txt",samplingRatio=0.1,prefersDecimal=True)
#primitivesAsString
df_json.printSchema()
df_json.show()
     

In [0]:
strt1="id int,name string,amt float, dop date,custom_corruptrow string"
df_json=spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/simple_json2.txt",allowComments=True,lineSep='~',allowSingleQuotes=True,allowUnquotedFieldNames=True,columnNameOfCorruptRecord="custom_corruptrow",schema=strt1,dateFormat='yyyy-dd-MM')
#primitivesAsString
df_json.printSchema()
df_json.show(10,False)
     

In [0]:
strt1="id int,name string,amt float, dop date,custom_corruptrow string"
df_json=spark.read.json("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/simple_json_multiline3.txt",allowComments=True,allowSingleQuotes=True,allowUnquotedFieldNames=True,columnNameOfCorruptRecord="custom_corruptrow",schema=strt1,dateFormat='yyyy-dd-MM',multiLine=True)
#primitivesAsString
df_json.printSchema()
df_json.show(10,False)

In [0]:
#I am converting the CSV data into ORC data format to achieve SCHEMA EVOLUTION(CHANGING)
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source1.txt",inferSchema=True,header=True).write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/orc_targetdata_merged/")
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source2.txt",inferSchema=True,header=True).write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/orc_targetdata_merged/",mode='append')
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source3.txt",inferSchema=True,header=True).write.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/orc_targetdata_merged/",mode='append')



In [0]:
orc_merged_data=spark.read.orc("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/orc_targetdata_merged/",mergeSchema=True)
display(orc_merged_data)

In [0]:

#I am converting the CSV data into ORC data format to achieve SCHEMA EVOLUTION(CHANGING)
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source1.txt",inferSchema=True,header=True).write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/parquet_targetdata_merged/")
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source2.txt",inferSchema=True,header=True).write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/parquet_targetdata_merged/",mode='append')
spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/source3.txt",inferSchema=True,header=True).write.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/parquet_targetdata_merged/",mode='append')


In [0]:
df_parquet=spark.read.parquet("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/SampleData/parquet_targetdata_merged/",mergeSchema=True)
display(df_parquet)